In [1]:
# https://docs.deepnote.com/importing-and-exporing/share-or-embed-a-cell

from pyproj import Transformer, CRS
# https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'colab'  # "browser" for pycharm, "notebook" for jupyter
import kaleido  # to render graphs in plotly as images
import numpy as np
import pandas as pd
from io import StringIO
import csv
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
# https://github.com/jupyter-widgets/ipywidgets/issues/3445

main_display = widgets.Output()

def show_it(inputs):
    with main_display:
        main_display.clear_output()
        display(list(inputs['new'].keys())[-1])
        print(list(inputs['new'].keys())[-1])
        print(' was successfully uploaded')

def clear():
    os.system('cls')

upload = widgets.FileUpload(
    accept='.xlsx',
    multiple=False)

upload.observe(show_it, names='_counter')
# if pyqt5 widgets fail:
# https://stackoverflow.com/questions/20790926/ipython-notebook-open-select-file-with-gui-qt-dialog
# https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/

In [ ]:
display(upload)

FileUpload(value={}, accept='.xlsx', description='Upload')

In [ ]:
# upload.value #---> for all the data regarding the uploaded file. + .values() generates the list of values
uploaded_filename = next(iter(upload.value))
print(uploaded_filename + ' was uploaded')

Pestalozzi_Koordinaten.xlsx was uploaded


In [ ]:
# import data into dataframe
input_file = list(upload.value.values())[0]
content = input_file['content']  # equal to upload.data[0]
if uploaded_filename.find('.xls') != -1:
    tbl = pd.read_excel(content, header=0, engine='openpyxl')
elif uploaded_filename.find('.csv') != -1:
    content = StringIO(content.decode('utf-8'))
    sniffer = csv.Sniffer()
    sniffer.preferred = [';', ',']
    dialect = sniffer.sniff(content)
    tbl = pd.read_csv(content, delimiter=dialect.delimiter, header=0)

# prepare dataframe
tbl = tbl.rename(columns={tbl.columns[0]: 'Pfahl', tbl.columns[1]: 'X', tbl.columns[2]: 'Y'})
tbl = tbl.iloc[0:,0:3]
# drop all but the X and Y columns
piles = tbl.Pfahl
tbl = tbl.astype({'X': 'float64', 'Y': 'float64'})
tbl.head()

,Pfahl,X,Y
0,BNO-01,3.452905e+06,5.526302e+06
1,BNO-02,3.452906e+06,5.526303e+06
2,BNO-03,3.452907e+06,5.526303e+06
3,BNO-04,3.452909e+06,5.526305e+06
4,BNO-05,3.452924e+06,5.526315e+06


In [ ]:
# plot raw unconverted data https://deepnote.com/@carlos-mendez/PYTHON-Plotly-Express-3e900d54-8143-405b-8be2-266157d4ec75
fig1 = px.scatter(tbl,
            x=tbl.columns[1],
            y=tbl.columns[2],
            #color="region",
            hover_name=tbl.columns[0],
            #animation_frame = tbl.columns[0]
)
fig1.show(renderer="colab")
#valid = False
#while not valid: #loop until the user enters a valid int
    #try:
        #num = int(input('Enter the number of points to be drawn: '))
        #valid = True #if this point is reached, x is a valid int
    #except ValueError:        print('Please only input digits')
#plt.scatter(x=tbl[tbl.columns[1]][0:num], y=tbl[tbl.columns[2]][0:num])
#plt.show()

In [ ]:
# https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/
print('Now choose a GK Zone. To determine if a GK zone is 2,3, 4 or 5,' +
    'look at the first number in the Easting/Rechtswert column')
print('First number in Easting is: ' + str(tbl['X'][0])[0])

def function(option):
    print(option)

w = widgets.Dropdown(
    options=['GK Zone 2 - epsg:31466', 'GK Zone 3 - epsg:31467',
    'GK Zone 4 - epsg:31468', 'GK Zone 5 - epsg:31469'],
    value=None,
    description='Source system:',
)

w.observe(
    lambda c: plot_content(c['new']) if (c['type'] == 'change' and c['name'] == 'value') else None
)

display(w)

Now choose a GK Zone. To determine if a GK zone is 2,3, 4 or 5,look at the first number in the Easting/Rechtswert column
First number in Easting is: 3


Dropdown(description='Source system:', options=('GK Zone 2 - epsg:31466', 'GK Zone 3 - epsg:31467', 'GK Zone 4…

In [ ]:
print('Now choose a GK Zone. To determine if a GK zone is 2,3, 4 or 5,' +
    'look at the first number in the Easting/Rechtswert column')
print('First number in Easting is: ' + str(tbl['X'][0])[0])

Now choose a GK Zone. To determine if a GK zone is 2,3, 4 or 5,look at the first number in the Easting/Rechtswert column
First number in Easting is: 3


In [ ]:
sourcesys = 'GK Zone 2 - epsg:31466'

In [ ]:
#if w.value != None:
#    source = w.value[w.value.find('epsg')::]
source = sourcesys[sourcesys.find('epsg')::]
proj_target = CRS.from_string("epsg:4326")  # wgs84
proj_source = CRS.from_string(source)  # source system
# for austrian system, Y and X are the normal ones.
# For german, Y=Hochwert is cartesian X, X=Rechtswert is cartesian Y
x = tbl['Y']; y = tbl['X']
transformer = Transformer.from_crs(proj_source, proj_target)
(nx, ny) = transformer.transform(x, y)
nx = np.array(nx)[np.newaxis]; nx = nx.T; ny = np.array(ny)[np.newaxis]; ny = ny.T
new_coord = pd.DataFrame(data=nx, index=piles, columns=['North']); new_coord['East'] = ny;
new_coord['Pfahl'] = ''; new_coord['Pfahl'].iloc[0:] = piles
new_coord = new_coord[['Pfahl','North', 'East']]
print(str(new_coord['North'].iloc[0]) + ',' + str(new_coord['East'].iloc[0]))
# plot data
fig2 = px.scatter(new_coord,
            x=new_coord.columns[2],
            y=new_coord.columns[1],
            #color="region",
            hover_name=new_coord.columns[0],
            #animation_frame = new_coord.columns[0]
)
fig2.show(renderer="colab")
#new_coord.plot(x='North', y='East', kind="scatter",colormap="YlOrRd")

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

49.87133359552492,8.343832778290398


In [ ]:
fig3 = px.scatter_mapbox(
    new_coord,  # Our DataFrame
    lat='North',
    lon='East',
    center={"lat": new_coord['North'].mean(), "lon": new_coord['East'].mean()},  # where map will be centered
    width=800,  # Width of map
    height=600,  # Height of map
    hover_data=['Pfahl'],  # what to display when hovering mouse over coordinate
    #color=new_coord['Koordinaten Art'],
    title='Karte'
)

fig3.update_layout(title_x=0.5, mapbox={'style': "open-street-map", 'center': {'lat': new_coord['North'].mean(),
                                                                   'lon': new_coord['East'].mean()}, 'zoom': 16},
                  hovermode='closest'
                  # annotations=[
                  #     dict(
                  #         x=0.1,
                  #         y=0.1,
                  #         align="right",
                  #         valign="top",
                  #         text='Koordinaten',
                  #         showarrow=False,
                  #         xref="paper",
                  #         yref="paper",
                  #         xanchor="center",
                  #         yanchor="top"
                  #     )
                  #]
                  )
# adding beautiful street layout to map and setting initial zoom

fig3.show(renderer="colab")
#https://plotly.com/python/renderers/

In [ ]:
import base64
import pandas as pd
from IPython.display import HTML

valid = False
while not valid: #loop until the user enters a valid int
    lang = input('Enter the language of your laptop (de or en): ')
    if lang == 'de':
        delimiter = ';'
        valid = True
    elif  lang == 'en':
        delimiter = ','
        valid = True
    else:
        valid = False # stay in the loop

def create_download_link(df, title = "Download converted coordinates", filename = "Converted_elements.csv"):
    csv = df.to_csv(index=False, sep=delimiter)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(new_coord)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a82d000-c111-4d9b-b2f0-364d0f452677' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>